In [8]:
"""A simple script to exercise the web app and celery to compute a result."""
from time import sleep

import requests
from qcelemental.models import AtomicInput, AtomicResult, Molecule
from qcelemental.models.common_models import Model

In [9]:

molecule = Molecule.from_data("pubchem:water")
model = Model(method="B3LYP", basis="6-31g")
driver = "energy"
atomic_input = AtomicInput(molecule=molecule, model=model, driver=driver)
print(atomic_input)

	Searching PubChem database for water (single best match returned)
	Found 1 result(s)
AtomicInput(driver='energy', model={'method': 'B3LYP', 'basis': '6-31g'}, molecule_hash='b6ec4fa')


In [15]:
atomic_input.molecule

NGLWidget()

In [16]:

# HOST = "http://localhost:8000"
HOST = "https://tcc.dev.mtzlab.com"

r1 = requests.post(
        f"{HOST}/compute",
        data=atomic_input.json(),
        params={"engine": "psi4"},
    )
print(r1.status_code)
print(r1.json())

200
6f995f37-667d-4a69-9695-e8482390fcd9


In [17]:
# Can run this cell a few times to check job status as work completes
r2 = requests.get(
        f"{HOST}/result",
        params={"job_id": r1.json()},
    )
print(r2)
print(r2.json())

<Response [200]>
['SUCCESS', {'id': None, 'schema_name': 'qcschema_output', 'schema_version': 1, 'molecule': {'schema_name': 'qcschema_molecule', 'schema_version': 2, 'validated': True, 'symbols': ['O', 'H', 'H'], 'geometry': [-0.10142170857663188, -0.07509332367630613, 0.0, -0.05369405559400764, 1.7554092274683715, 0.0, 1.6633301703714394, -0.5636236870312887, 0.0], 'name': 'IUPAC oxidane', 'molecular_charge': 0.0, 'molecular_multiplicity': 1, 'fix_com': False, 'fix_orientation': False, 'provenance': {'creator': 'QCElemental', 'version': 'v0.17.0', 'routine': 'qcelemental.molparse.from_string'}}, 'driver': 'energy', 'model': {'method': 'B3LYP', 'basis': '6-31g'}, 'keywords': {}, 'protocols': {}, 'extras': {'qcvars': {'-D ENERGY': 0.0, 'CURRENT DIPOLE X': 1.9802882827275228, 'CURRENT DIPOLE Y': 1.4662172058184162, 'CURRENT DIPOLE Z': 0.0, 'CURRENT ENERGY': -76.38546709866571, 'CURRENT REFERENCE ENERGY': -76.38546709866571, 'DFT FUNCTIONAL TOTAL ENERGY': -76.38546709866571, 'DFT TOTAL E

In [18]:
# Print out final results
result = AtomicResult(**r2.json()[1])
print(result)
print(result.return_result)

AtomicResult(driver='energy', model={'method': 'B3LYP', 'basis': '6-31g'}, molecule_hash='04bca57')
-76.38546709866571


In [23]:
from pprint import pprint
pprint(result.stdout)

('\n'
 '  Memory set to 102.900 GiB by Python driver.\n'
 '\n'
 '*** tstart() called on 37e993f6cff6\n'
 '*** at Fri Nov 20 21:15:53 2020\n'
 '\n'
 '   => Loading Basis Set <=\n'
 '\n'
 '    Name: 6-31G\n'
 '    Role: ORBITAL\n'
 '    Keyword: BASIS\n'
 '    atoms 1   entry O          line   117 file '
 '/opt/conda/share/psi4/basis/6-31g.gbs \n'
 '    atoms 2-3 entry H          line    26 file '
 '/opt/conda/share/psi4/basis/6-31g.gbs \n'
 '\n'
 '\n'
 '         ---------------------------------------------------------\n'
 '                                   SCF\n'
 '               by Justin Turney, Rob Parrish, Andy Simmonett\n'
 '                          and Daniel G. A. Smith\n'
 '                              RKS Reference\n'
 '                       10 Threads, 105369 MiB Core\n'
 '         ---------------------------------------------------------\n'
 '\n'
 '  ==> Geometry <==\n'
 '\n'
 '    Molecular point group: cs\n'
 '    Full point group: Cs\n'
 '\n'
 '    Geometry (in Bohr),

['id',
 'schema_name',
 'schema_version',
 'molecule',
 'driver',
 'model',
 'keywords',
 'protocols',
 'extras',
 'provenance',
 'properties',
 'wavefunction',
 'return_result',
 'stdout',
 'stderr',
 'success',
 'error',
 '__config__',
 '__fields__',
 '__validators__',
 '__pre_root_validators__',
 '__post_root_validators__',
 '__schema_cache__',
 '__json_encoder__',
 '__custom_root_type__',
 '__private_attributes__',
 '__slots__',
 '__module__',
 '__annotations__',
 '_input_to_output',
 '_validate_return_result',
 '_wavefunction_protocol',
 '_stdout_protocol',
 '__doc__',
 '__base_doc__',
 '__model_autodoc_applied__',
 '__abstractmethods__',
 '_abc_impl',
 '__signature__',
 '__repr_args__',
 '__repr__',
 '__str__',
 'Config',
 '__init_subclass__',
 'parse_raw',
 'parse_file',
 'dict',
 'serialize',
 'json',
 'compare',
 '__init__',
 '__setattr__',
 '__getstate__',
 '__setstate__',
 '_init_private_attributes',
 'parse_obj',
 'from_orm',
 'construct',
 'copy',
 'schema',
 'schema_json'

In [28]:
result.stderr